## Chapter 2: Your First .NET Aspire Application

In Chapter 1, we introduced the challenges of distributed development and gave you a high‑level overview of .NET Aspire. You even ran your first Aspire application using the starter template. Now it’s time to slow down and really understand what’s inside that template. By the end of this chapter, you’ll know the purpose of each project, how they fit together, and how to navigate the powerful Aspire Dashboard. You’ll also modify the service defaults to add a custom health check—your first hands‑on customization.

---

### 2.1 Creating a New Aspire Project from a Template

Let’s create a fresh Aspire solution so we can examine every file. Open a terminal and run:

```bash
dotnet new aspire-starter -n MyAspireApp
```

This command uses the `aspire-starter` template to generate a solution named `MyAspireApp`. The template includes four projects:

- `MyAspireApp.AppHost`
- `MyAspireApp.ServiceDefaults`
- `MyAspireApp.ApiService`
- `MyAspireApp.Web`

Navigate into the solution folder:

```bash
cd MyAspireApp
```

Now open the solution in your editor. If you’re using Visual Studio Code, type `code .`; if you’re using Visual Studio, open the `.sln` file.

---

### 2.2 Anatomy of an Aspire Solution

Let’s explore each project and its role.

#### 2.2.1 The AppHost Project

The AppHost is the orchestrator. It defines all the resources that make up your distributed application. Open `MyAspireApp.AppHost/Program.cs`:

```csharp
using Aspire.Hosting;

var builder = DistributedApplication.CreateBuilder(args);

var apiService = builder.AddProject<Projects.MyAspireApp_ApiService>("apiservice");

builder.AddProject<Projects.MyAspireApp_Web>("webfrontend")
    .WithReference(apiService);

builder.Build().Run();
```

**Explanation**:
- `DistributedApplication.CreateBuilder(args)` initializes the Aspire builder. This sets up the infrastructure to model and run your application.
- `builder.AddProject<T>("apiservice")` registers a .NET project as a resource. The type `Projects.MyAspireApp_ApiService` is a generated class that points to the API project. The string `"apiservice"` is the **logical name** that other resources will use to refer to this service.
- `builder.AddProject<...>("webfrontend").WithReference(apiService)` adds the web frontend and establishes a dependency on the API service. `WithReference` does two important things:
  1. It tells Aspire to inject the API’s endpoint information into the web frontend’s environment variables and configuration.
  2. It ensures the API service is started before the web frontend (order is respected during launch).
- Finally, `builder.Build().Run()` builds the application model and starts all resources.

The AppHost project also contains a `Properties/launchSettings.json` that defines profiles for running the AppHost, typically with the Aspire Dashboard enabled.

#### 2.2.2 The ServiceDefaults Project

The ServiceDefaults project is a class library that contains shared configuration for all your services. Open `MyAspireApp.ServiceDefaults/Extensions.cs`. This file defines an extension method `AddServiceDefaults` that every service calls in its `Program.cs`.

```csharp
using Microsoft.AspNetCore.Builder;
using Microsoft.AspNetCore.Diagnostics.HealthChecks;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.Extensions.Diagnostics.HealthChecks;
using Microsoft.Extensions.Logging;
using OpenTelemetry;
using OpenTelemetry.Metrics;
using OpenTelemetry.Trace;

namespace Microsoft.Extensions.Hosting;

public static class Extensions
{
    public static IHostApplicationBuilder AddServiceDefaults(this IHostApplicationBuilder builder)
    {
        builder.ConfigureOpenTelemetry();

        builder.AddDefaultHealthChecks();

        builder.Services.AddServiceDiscovery();

        builder.Services.ConfigureHttpClientDefaults(http =>
        {
            // Turn on resilience by default
            http.AddStandardResilienceHandler();

            // Turn on service discovery by default
            http.AddServiceDiscovery();
        });

        return builder;
    }

    public static IHostApplicationBuilder ConfigureOpenTelemetry(this IHostApplicationBuilder builder)
    {
        builder.Logging.AddOpenTelemetry(logging =>
        {
            logging.IncludeFormattedMessage = true;
            logging.IncludeScopes = true;
        });

        builder.Services.AddOpenTelemetry()
            .WithMetrics(metrics =>
            {
                metrics.AddAspNetCoreInstrumentation()
                       .AddHttpClientInstrumentation()
                       .AddRuntimeInstrumentation();
            })
            .WithTracing(tracing =>
            {
                tracing.AddAspNetCoreInstrumentation()
                       .AddHttpClientInstrumentation()
                       .AddEntityFrameworkCoreInstrumentation(); // If you use EF Core
            });

        builder.Services.AddOpenTelemetryExporters();

        return builder;
    }

    private static IHostApplicationBuilder AddOpenTelemetryExporters(this IHostApplicationBuilder builder)
    {
        var useOtlpExporter = !string.IsNullOrWhiteSpace(builder.Configuration["OTEL_EXPORTER_OTLP_ENDPOINT"]);
        if (useOtlpExporter)
        {
            builder.Services.AddOpenTelemetry().UseOtlpExporter();
        }

        return builder;
    }

    public static IHostApplicationBuilder AddDefaultHealthChecks(this IHostApplicationBuilder builder)
    {
        builder.Services.AddHealthChecks()
            // Add a default liveness check to ensure app is responsive
            .AddCheck("self", () => HealthCheckResult.Healthy(), ["live"]);

        return builder;
    }
}
```

**Explanation of key parts**:
- **`ConfigureOpenTelemetry`**:
  - Enables OpenTelemetry logging, metrics, and tracing.
  - Adds instrumentation for ASP.NET Core, HTTP clients, and runtime metrics.
  - The exporters are configured to send telemetry to the OTLP endpoint (provided by the Aspire Dashboard) if the `OTEL_EXPORTER_OTLP_ENDPOINT` environment variable is set.
- **`AddDefaultHealthChecks`**:
  - Registers a simple “self” health check that always returns healthy. This is a liveness probe.
  - The `["live"]` tag marks it for Kubernetes liveness probes, but it’s also used by the Aspire Dashboard.
- **Service Discovery**:
  - `builder.Services.AddServiceDiscovery()` registers the service discovery infrastructure.
  - The HTTP client configuration adds a `ServiceDiscoveryHandler` that resolves logical names (like `http://apiservice`) to actual addresses using the endpoints injected by the AppHost.
- **Resilience**:
  - `http.AddStandardResilienceHandler()` adds a Polly-based resilience handler with default retry, timeout, and circuit‑breaker policies. This makes your HTTP calls more robust against transient failures.

The ServiceDefaults project is referenced by both `ApiService` and `Web`. By calling `AddServiceDefaults()`, each service gets all these features automatically.

#### 2.2.3 The ApiService Project

This is a minimal API project. Open `MyAspireApp.ApiService/Program.cs`:

```csharp
using MyAspireApp.ServiceDefaults;

var builder = WebApplication.CreateBuilder(args);

// Add service defaults
builder.AddServiceDefaults();

// Add services to the container.
builder.Services.AddProblemDetails();

var app = builder.Build();

// Configure the HTTP request pipeline.
app.UseExceptionHandler();

var summaries = new[]
{
    "Freezing", "Bracing", "Chilly", "Cool", "Mild", "Warm", "Balmy", "Hot", "Sweltering", "Scorching"
};

app.MapGet("/weatherforecast", () =>
{
    var forecast = Enumerable.Range(1, 5).Select(index =>
        new WeatherForecast
        (
            DateOnly.FromDateTime(DateTime.Now.AddDays(index)),
            Random.Shared.Next(-20, 55),
            summaries[Random.Shared.Next(summaries.Length)]
        ))
        .ToArray();
    return forecast;
});

app.Run();

record WeatherForecast(DateOnly Date, int TemperatureC, string? Summary)
{
    public int TemperatureF => 32 + (int)(TemperatureC / 0.5556);
}
```

**Explanation**:
- `builder.AddServiceDefaults()` brings in OpenTelemetry, health checks, and service discovery.
- `builder.Services.AddProblemDetails()` adds support for Problem Details (RFC 7807) responses, which is good practice for APIs.
- The rest is a standard minimal API endpoint returning a weather forecast.

Notice that there’s no explicit connection to the web frontend. The API simply runs and exposes endpoints.

#### 2.2.4 The Web Project

This is a Blazor web application. Open `MyAspireApp.Web/Program.cs`:

```csharp
using MyAspireApp.ServiceDefaults;
using MyAspireApp.Web;
using MyAspireApp.Web.Components;

var builder = WebApplication.CreateBuilder(args);

// Add service defaults
builder.AddServiceDefaults();

// Add services to the container.
builder.Services.AddRazorComponents()
    .AddInteractiveServerComponents();

// Register an HTTP client for the API
builder.Services.AddHttpClient<WeatherApiClient>(client =>
    {
        // This URL uses "http://apiservice" which is the logical name of the API service
        client.BaseAddress = new Uri("http://apiservice");
    });

var app = builder.Build();

// Configure the HTTP request pipeline.
if (!app.Environment.IsDevelopment())
{
    app.UseExceptionHandler("/Error", createScopeForErrors: true);
    app.UseHsts();
}

app.UseHttpsRedirection();
app.UseStaticFiles();
app.UseAntiforgery();

app.MapRazorComponents<App>()
    .AddInteractiveServerRenderMode();

app.Run();
```

**Explanation**:
- Again, `AddServiceDefaults()` configures telemetry, health checks, and service discovery.
- `AddHttpClient<WeatherApiClient>` registers an HTTP client typed to `WeatherApiClient`. The base address is set to `http://apiservice`. Because service discovery is enabled, this logical name will be resolved to the actual URL of the API service (e.g., `http://localhost:5001`).
- The `WeatherApiClient` class (located in `MyAspireApp.Web/WeatherApiClient.cs`) uses this client to call the API:

```csharp
namespace MyAspireApp.Web;

public class WeatherApiClient(HttpClient httpClient)
{
    public async Task<WeatherForecast[]> GetWeatherAsync()
    {
        return await httpClient.GetFromJsonAsync<WeatherForecast[]>("/weatherforecast") ?? [];
    }
}

public record WeatherForecast(DateOnly Date, int TemperatureC, string? Summary)
{
    public int TemperatureF => 32 + (int)(TemperatureC / 0.5556);
}
```

This client is then used in a Blazor component (`Components/Pages/Weather.razor`) to display the forecast.

---

### 2.3 Running the AppHost and Seeing the Aspire Dashboard

Now let’s run the application. In the terminal, go to the AppHost directory:

```bash
cd MyAspireApp.AppHost
dotnet run
```

After a few seconds, you’ll see output like:

```
...
Now listening on: http://localhost:15000
Login to the dashboard at http://localhost:15000/login?t=...
...
```

Open the provided URL. You’ll be prompted to log in (the token is in the URL). After logging in, the Aspire Dashboard appears.

#### 2.3.1 The Resources Page

The **Resources** tab shows a list of all resources defined in the AppHost. You’ll see:

- **apiservice** – The API project. Its state should be “Running”. Clicking on it reveals its endpoint(s) and environment variables.
- **webfrontend** – The web project, also “Running”, with its endpoint.
- The **Dashboard** itself is also listed as a resource.

This view gives you a bird’s‑eye view of your entire distributed application. You can see which services are up, their URLs, and even restart individual resources.

#### 2.3.2 The Console Logs Page

Click on the **Console Logs** tab. Here you see structured logs from all services. You can filter by service (e.g., show only logs from `apiservice`), by log level, or search for specific text. The logs include timestamps and are correlated with traces, so you can jump from a log entry to the corresponding trace.

#### 2.3.3 The Traces Page

The **Traces** tab is where distributed tracing shines. Every incoming HTTP request generates a trace. For example, when you refresh the web frontend’s weather page, a request goes to the web frontend, which then calls the API. In the traces view, you’ll see a single trace representing that entire operation, with two spans:

- One span for the web frontend handling the request.
- One span for the HTTP call to the API.

You can click on a trace to see its timeline, duration, and any errors. This is invaluable for understanding performance bottlenecks and debugging failures across service boundaries.

#### 2.3.4 The Metrics Page

The **Metrics** tab displays real‑time performance metrics: request rates, error rates, duration histograms, and process metrics like CPU and memory usage. These are collected via OpenTelemetry and can be used to monitor the health of your application during development.

---

### 2.4 Deep Dive into Service Defaults

Now that you’ve seen the dashboard, let’s revisit the service defaults and understand exactly what they provide.

#### 2.4.1 OpenTelemetry Integration

OpenTelemetry is the industry standard for observability. In `ConfigureOpenTelemetry`, we enable:

- **Logging**: `builder.Logging.AddOpenTelemetry` captures structured logs and sends them to the OTLP exporter.
- **Metrics**: `WithMetrics` adds instrumentations for ASP.NET Core (`AddAspNetCoreInstrumentation`), HTTP client calls (`AddHttpClientInstrumentation`), and runtime metrics (`AddRuntimeInstrumentation`). These metrics are collected and exported.
- **Tracing**: `WithTracing` adds similar instrumentations for tracing, including Entity Framework Core if you use it.

The `AddOpenTelemetryExporters` method checks for the `OTEL_EXPORTER_OTLP_ENDPOINT` environment variable (set by the AppHost) and configures the OTLP exporter accordingly. This sends all telemetry to the Aspire Dashboard.

#### 2.4.2 Health Checks

`AddDefaultHealthChecks` registers a simple health check named “self” that always returns healthy. It also adds a tag “live”. In production, you might have more sophisticated checks (database connectivity, etc.). The health check endpoint is automatically exposed at `/health` by the Aspire integration.

The dashboard polls these health endpoints and displays the status in the Resources view. A service that fails its health check will be marked as unhealthy.

#### 2.4.3 Service Discovery

`AddServiceDiscovery` registers the core service discovery mechanism. When you call `AddHttpClient` and set a base address to a logical name like `http://apiservice`, the `ServiceDiscoveryHandler` intercepts the request and resolves the actual URL. It does this by reading environment variables injected by the AppHost. For example, the AppHost sets an environment variable like `services__apiservice__http__0` to `http://localhost:5001`. The handler picks this up and routes requests accordingly.

#### 2.4.4 Resilience

The `AddStandardResilienceHandler` extension adds a Polly pipeline with:

- **Retry**: Automatically retries failed requests (with exponential backoff) for certain HTTP status codes (like 5XX) and network exceptions.
- **Timeout**: Prevents requests from hanging indefinitely.
- **Circuit breaker**: Stops sending requests to a failing endpoint temporarily to give it time to recover.

These policies make your application more resilient to transient failures—exactly what you need in a distributed environment.

---

### 2.5 Hands-on: Adding a Custom Health Check

Let’s put your knowledge into practice. We’ll add a custom health check to the API service that verifies connectivity to a dummy “database”. This will demonstrate how to extend the service defaults and see the result in the dashboard.

#### Step 1: Add a health check class

In the `ApiService` project, create a new folder `HealthChecks` and add a class `DatabaseHealthCheck.cs`:

```csharp
using Microsoft.Extensions.Diagnostics.HealthChecks;

namespace MyAspireApp.ApiService.HealthChecks;

public class DatabaseHealthCheck : IHealthCheck
{
    public Task<HealthCheckResult> CheckHealthAsync(
        HealthCheckContext context,
        CancellationToken cancellationToken = default)
    {
        // Simulate a database check
        bool databaseIsAvailable = true; // Assume it's always healthy for now

        if (databaseIsAvailable)
        {
            return Task.FromResult(HealthCheckResult.Healthy("Database is reachable"));
        }

        return Task.FromResult(HealthCheckResult.Unhealthy("Database is unreachable"));
    }
}
```

This simple check always returns healthy, but in a real scenario you might try to open a connection to a database.

#### Step 2: Register the health check in the API’s Program.cs

Open `ApiService/Program.cs` and modify it to register the custom health check. Since the service defaults already added a health check builder, we can just add ours.

```csharp
using MyAspireApp.ApiService.HealthChecks; // Add this

var builder = WebApplication.CreateBuilder(args);

// Add service defaults
builder.AddServiceDefaults();

// Add custom health check
builder.Services.AddHealthChecks()
    .AddCheck<DatabaseHealthCheck>("database");

// ... rest of the code
```

**Important**: The call to `AddHealthChecks()` returns an `IHealthChecksBuilder`. We add our custom check with a name `"database"`. The existing “self” check is still there because `AddServiceDefaults` already added it.

#### Step 3: Run the application and observe

Run the AppHost again (`dotnet run` in the AppHost directory). Open the dashboard and go to the **Resources** page. Click on the `apiservice` resource. You should see its health status now includes two checks: “self” and “database”. Both should be healthy.

If you want to simulate failure, modify `DatabaseHealthCheck` to return unhealthy and see the dashboard reflect that.

#### Step 4: (Optional) Add a health check endpoint for the database

The default `/health` endpoint already aggregates all registered health checks. You can test it by navigating to `http://localhost:5001/health` (replace the port with your API’s actual port). You’ll get a JSON response with the overall status and details of each check.

---

### 2.6 Summary

In this chapter, we thoroughly dissected the Aspire starter template. You learned:

- The purpose of each project: AppHost (orchestrator), ServiceDefaults (shared configuration), ApiService (backend), and Web (frontend).
- How service discovery and resilience are configured in the defaults.
- How to navigate the Aspire Dashboard to monitor logs, traces, metrics, and resource health.
- How to extend the service defaults by adding a custom health check.

You’re now comfortable with the basic structure and can start building more complex applications. In the next chapter, we’ll dive into the AppHost project in detail and learn how to add real infrastructure—like a PostgreSQL database—using .NET Aspire components.

---

**Exercises**

1. Modify the `DatabaseHealthCheck` to randomly return unhealthy 10% of the time. Observe how the dashboard updates.
2. In the web frontend, add a second HTTP client that calls a different endpoint of the API (e.g., `/products`). Implement the endpoint in the API and display the data in a new Blazor page.
3. Explore the traces tab. Generate some load by refreshing the weather page rapidly and examine the trace details. Identify which part of the request took the longest.

In the next chapter, we’ll integrate a PostgreSQL database using an Aspire component, and you’ll see how Aspire simplifies connection management even further.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='1. welcome_to_the_distributed_world.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='3. the_service_defaults_project.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
